In [2]:
import os
import random
import cv2
import numpy as np
import cv2
import hashlib
import uuid
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from skimage import io, color, filters, transform, exposure
from pencil_texture import *
from crop_sketch_query import sketch_crop

In [4]:
def gen_image_name():
    id = np.random.randint(1, 255)
    current_time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    full_name = current_time + '_' + str(id) + '_' + str(uuid.uuid4())
    encrypted = hashlib.sha256(full_name.encode()).hexdigest()
    return encrypted[:16]

def random_flip(img):
    if np.random.rand() < 0.5:
        img = cv2.flip(img, 1)
    return img

def random_rotate(img):
    if np.random.rand() < 0.7:
        # Randomly rotate the image in a small angle
        angle = np.random.randint(-10, 10)
        rows, cols = img.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        img = cv2.warpAffine(img, M, (cols, rows), flags=cv2.INTER_CUBIC, borderMode = cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return img

In [9]:
OBJECT_DIR = './SketchANIMAR2023/3D_Model_References/References'
RINGVIEW_DIR = './SketchANIMAR2023/3D_Model_References/generated_models'

CANNY_DIR = './SketchANIMAR2023/3D_Model_References/generated_sketches'
ARTLINE_DIR = './SketchANIMAR2023/3D_Model_References/artline_generated_sketches'

NEW_QUERY_DIR = './SketchANIMAR2023/Train/NewSketchQuery_Train_3'
df_train = pd.read_csv('./csv/train_skt.csv')
df_test = pd.read_csv('./csv/test_skt.csv')

os.makedirs(NEW_QUERY_DIR, exist_ok=True)

for object in tqdm(os.listdir(OBJECT_DIR)):
    # used_rings = [2, 3, 4]
    used_rings = [3]

    object_name = object.split('.')[0]
    common_ring_folder_path = os.path.join(
        RINGVIEW_DIR, 'ring{}', object_name, 'render')
    common_canny_folder_path = os.path.join(
        CANNY_DIR, 'ring{}', object_name, 'render')
    common_artline_folder_path = os.path.join(
        ARTLINE_DIR, 'ring{}', object_name, 'render')

    for used_ring in used_rings:
        ring_folder_path = common_ring_folder_path.format(used_ring)
        canny_folder_path = common_canny_folder_path.format(used_ring)
        artline_folder_path = common_artline_folder_path.format(used_ring)

        for _ in range(3):
            random_image = random.choice(sorted(os.listdir(ring_folder_path))[2:11])

            options = ['canny', 'artline', 'pencil']
            probabilities = (0.6, 0.4, 0.0)
            option = random.choices(options, weights=probabilities)[0]

            if option == 'canny':
                img = cv2.imread(os.path.join(canny_folder_path, random_image))
            elif option == 'artline':
                img = cv2.imread(os.path.join(
                    artline_folder_path, random_image))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            else:
                raw_img = io.imread(os.path.join(canny_folder_path, random_image))
                pencil_tex = './pencil_textures/pencil1.jpg'
                img = gen_pencil_drawing(raw_img, kernel_size=8, stroke_width=1, num_of_directions=8, smooth_kernel="gauss",
                                        gradient_method=1, rgb=True, w_group=2, pencil_texture_path=pencil_tex,
                                        stroke_darkness=2, tone_darkness=0)
                img[img > 1] = 1
                img[img < -1] = -1
                img = exposure.rescale_intensity(img, in_range=(0, 1)).astype(np.float32)
                # Convert the skimage image to OpenCV format
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            img = random_flip(img)
            img = random_rotate(img)
            try:
                cropped_img = sketch_crop(img)
            except:
                cropped_img = img

            img = cropped_img
            img_name = gen_image_name()
            cv2.imwrite(os.path.join(NEW_QUERY_DIR, img_name + '.jpg'), img)

            new_data = {'obj_id': object_name, 'sketch_id': img_name,
                        'obj_filename': object_name + '.obj', 'sket_filename': img_name + '.jpg'}

            # add to train/val
            p = np.random.rand()
            if p < 0.15:        
                df_test = df_test.append(new_data, ignore_index=True)
            else:
                df_train = df_train.append(new_data, ignore_index=True)

df_train.to_csv(os.path.join('./csv', 'new_train_skt_3.csv'), index=False)
df_test.to_csv(os.path.join('./csv', 'new_test_skt_3.csv'), index=False)

100%|██████████| 711/711 [00:29<00:00, 23.73it/s]


In [10]:
import shutil

orig_query_folder = './SketchANIMAR2023/Train/CroppedSketchQuery_Train/'

for query in os.listdir(orig_query_folder):
    shutil.copy(os.path.join(orig_query_folder, query), os.path.join(NEW_QUERY_DIR, query))

In [11]:
!ls $NEW_QUERY_DIR | wc -l

2192


In [19]:
df_train = pd.read_csv(os.path.join('./csv', 'new_train_skt_3.csv'))
df_test = pd.read_csv(os.path.join('./csv', 'new_test_skt_3.csv'))

In [22]:
df_train['sket_filename'].unique().shape

(1914,)

In [23]:
df_test['sket_filename'].unique().shape

(278,)

In [20]:
all_skts = os.listdir(NEW_QUERY_DIR)

train_skts = df_train['sket_filename'].unique()
for skt in train_skts:
    if skt not in all_skts:
        print(skt)

In [21]:
test_skts = df_test['sket_filename'].unique()
for skt in test_skts:
    if skt not in all_skts:
        print(skt)